In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import os
from os import path
import sys
import pickle
from Bio import SeqIO

In [5]:
# this function is copied from HMM.ipynb
# array: numpy array
# flank: positive integer
def sliding_window(array, flank):
    assert flank > 0
    assert type(array) is np.ndarray
    assert np.logical_not(np.isnan(np.sum(array)))
    nrow = array.shape[0]
    assert nrow > 0
    ncol = array.shape[1]
    assert ncol > 0
    res = np.empty(shape=(nrow, (2*flank+1)*ncol))
    res[:] = np.nan
    for i in list(range(0,nrow)):
        s, e = i-flank, i+flank+1
        k = 0;
        for j in list(range(s,e)):
            if (j < 0 or j >= nrow):
                res[i, k:k+ncol] = 0
            else:
                assert np.logical_not(np.isnan(np.sum(array[j])))
                assert array[j].shape == (ncol,)
                res[i, k:k+ncol] = array[j]
            k += ncol
    assert np.logical_not(np.isnan(np.sum(res)))
    assert res.shape == (nrow, (2*flank+1)*ncol)
    return res

# this function rounds predictions into 1 and 0s
def argmax(arr):
    n, c = arr.shape
    assert c == 3
    assert type(arr) is np.ndarray
    assert np.logical_not(np.isnan(np.sum(arr)))
    res = np.empty(shape=(n,c))
    res[:] = np.nan
    for i in list(range(0,n)):
        max_idx = np.argmax(arr[i])
        if max_idx == 0:
            res[i] = np.array([1, 0, 0])
        elif max_idx == 1:
            res[i] = np.array([0, 1, 0])
        else:
            assert max_idx == 2
            res[i] = np.array([0, 0, 1])
    assert np.logical_not(np.isnan(np.sum(res)))
    return res

def get_dssp(ID):
    path = '/homes/adrozdetskiy/Projects/JnetDatasets/DSSP_out/' + ID + '.sec'
    ls = list(pd.read_csv(path).loc[0].values)
    ls[0] = ls[0][-1:] # remove the DSSP: part from first list item
    res = np.empty(shape=(len(ls),3))
    res[:] = np.nan
    for i in range(0,len(ls)):
        if ls[i] == 'H':
            res[i] = np.array([1,0,0])
        else:
            if ls[i] == 'E' or ls[i] == 'B':
                res[i] = np.array([0,1,0])
            else:
                assert ls[i] != None
                res[i] = np.array([0,0,1])
    assert not np.isnan(np.sum(res))
    return res

def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)
    
# given the prediction and truth for a sequence
# write a function to calculate 0 Hcorrect, 1 Htotal, 2 Ecorrect, 3 Etotal, 4 Ccorrect, 5 Ctotal, 6 Ncorrect, 7 Ntotal
# and return it as a np array
# e.g. seqID 21690
# require numpy as np
def get_Qall(pred_ndarr, truth_ndarr):
    assert type(pred_ndarr)==np.ndarray
    assert type(truth_ndarr)==np.ndarray
    assert pred_ndarr.shape==truth_ndarr.shape

    # subroutine to update a np.ndarray of shape (1,8) given a single digit prediction
    def update(pred_arr,truth_arr,res_arr):
        pred = np.argmax(pred_arr)
        truth = np.argmax(truth_arr)
        if pred==truth:
            res_arr[0,6] +=1 # increment Ncorrect
            res_arr[0,pred+truth] +=1 # increment H/E/C-correct
            res_arr[0,2*truth+1] +=1 # increment H/E/C-total
        else:
            res_arr[0,2*truth+1] += 1 # increment H/E/C-total
        res_arr[0,7] += 1 # increment Ntotal
        return res_arr
    
    results = np.zeros((1,8))
    for (pred_arr, truth_arr) in zip(pred_ndarr, truth_ndarr):
        results = update(pred_arr, truth_arr, results)
    
    assert results[0,0]+results[0,2]+results[0,4]==results[0,6]
    assert results[0,1]+results[0,3]+results[0,5]==results[0,7]
    
    return results

#convert (1,3) to a character. works for both dssp and prediction
def collapse(arr):
    assert type(arr)==np.ndarray
    i = np.argmax(arr)
    assert i<3;
    assert i>=0;
    if (i==0):
        return 'H'
    elif (i==1):
        return 'E'
    else:
        return 'C'

# Create SOV files for HMM 2-layer model

## Save predictions for all 7 folds

In [3]:
# first make sure that output path is valid, otherwise computation will go to waste
out_paths = ["/cluster/gjb_lab/2472402/outputs/hmm_cross_val/sov/cv%d/" % num for num in range(1,8)]
assert all([path.exists(out_path) & (out_path[-1] == '/') for out_path in out_paths])

# path to training and validation examples
paths = ["/homes/adrozdetskiy/Projects/JnetDatasets/Jnet_training_output_v2/cross-val%d/" % num for num in range(1,8)]

# sequence dictionary or sd. need this to retrieve dssp information
sd = pickle.load(open('/cluster/gjb_lab/2472402/data/cross-val/cross_val_dict.pkl','rb')) 


# for each fold in the 7 fold cross validation procedure, do...
for counter, (out_path,p) in enumerate(zip(out_paths,paths)):

    counter += 1 # start from 1
    
    valid_path = p + 'valid/'
    
    # get file names ending with .hmm
    valid_files = [f for f in os.listdir(valid_path) if f[-4:]=='.hmm']
    
    # read profile hmms as numpy arrays
    valid_hmm = [np.genfromtxt(fname = valid_path + fn) for fn in valid_files]
    
    # window profile hmms to get patterns
    # layer 1 sliding window flank = 8
    X_valid = [sliding_window(hmm, flank=8) for hmm in valid_hmm]
    del valid_hmm
    
    # get DSSP information 
    
    # remove .hmm extension and convert to int for lookup
    valid_numbers = [int(fn[:-4]) for fn in valid_files] 
    
    # lookup seqIDs given jnet number
    valid_dssp_IDs = [sd[sd.number == num].letters.values[0] for num in valid_numbers]
    
    # obtain 3-column DSSP of domains given their seqID. See get_dssp() defined above
    Y_valid = [get_dssp(ID) for ID in valid_dssp_IDs]

    assert all([y.shape[1] == 3 for y in Y_valid])
    assert all([x.shape[0] == y.shape[0] for (x, y) in zip(X_valid, Y_valid)])
    
    # generate layer 1 predictions
    
    model1 = keras.models.load_model('/cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold%d_model1' % counter)

    Y_pred_1 = [argmax(model1.predict(X)) for X in X_valid]
    
    # convert layer 1 predictions into layer 2 input
    X_valid_2 = [sliding_window(Y, flank = 9) for Y in Y_pred_1]
    
    # discard model object (and its input) from RAM
    del model1
    del Y_pred_1
    
    # sanity checks before passing into model2
    assert all([X.shape[1] == 57 for X in X_valid_2])
    assert all([X.dtype == 'float64' for X in X_valid_2])
    
    # generate layer 2 predictions
    
    model2 = keras.models.load_model('/cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold%d_model2' % counter)

    Y_pred_2 = [model2.predict(X) for X in X_valid_2]
    
    # discard model object (and its input) from RAM
    del model2
    del X_valid_2
    
    # write SOV input into file
    # read fasta sequence
    
    pickle.dump([valid_numbers, valid_dssp_IDs, Y_valid, Y_pred_2], open(out_path+"input.pkl", 'wb'))
    print("Finished saving predictions for fold %d\n" % counter)

Finished saving predictions for fold 1

Finished saving predictions for fold 2

Finished saving predictions for fold 3

Finished saving predictions for fold 4

Finished saving predictions for fold 5

Finished saving predictions for fold 6

Finished saving predictions for fold 7



Replace pickle part with this 

In [ ]:
    for (number, seqID, dssp, ypred) in list(zip(valid_numbers, valid_dssp_IDs, Y_valid, Y_pred_2)):

        # read fasta sequence
        fastafile = '/cluster/gjb_lab/2472402/data/retr231/training/' + seqID + '.fasta'
        seq = list(SeqIO.parse(open(fastafile),'fasta'))[0] # convert iterator to list then take element 0

        # write to SOV file
        outfile = out_path + str(number) + '.sov.in'
        with open(outfile, 'w') as f:
            # write header line
            f.write('AA  OSEC PSEC\n')
            # write each line
            for (aa, osec_arr, psec_arr) in zip(seq, dssp, ypred):
                osec = collapse(osec_arr)
                psec = collapse(psec_arr)
                f.write("%s   %s    %s\n" % (aa, osec, psec))

    print("Finished writing SOV files for cross-val fold %d\n" % counter)
    

## Output predictions in SOV format

In [1]:
import numpy as np
import pandas as pd
import os
from os import path
import sys
import pickle
from Bio import SeqIO

In [6]:
infiles = ["/cluster/gjb_lab/2472402/outputs/hmm_cross_val/sov/cv%d/input.pkl" % num for num in range(1,8)]
[valid_numbers, valid_dssp_IDs, Y_valid, Y_pred_2] = pickle.load(open(infiles[0],'rb'))
out_path = "/cluster/gjb_lab/2472402/outputs/hmm_cross_val/sov/tmp/"

for (number, seqID, dssp, ypred) in list(zip(valid_numbers, valid_dssp_IDs, Y_valid, Y_pred_2)):

    # read fasta sequence
    fastafile = '/cluster/gjb_lab/2472402/data/retr231/training/' + seqID + '.fasta'
    seq = list(SeqIO.parse(open(fastafile),'fasta'))[0] # convert iterator to list then take element 0

    # write to SOV file
    outfile = out_path + str(number) + '.sov.in'
    with open(outfile, 'w') as f:
        # write header line
        f.write('AA  OSEC PSEC\n')
        # write each line
        for (aa, osec_arr, psec_arr) in zip(seq, dssp, ypred):
            osec = collapse(osec_arr)
            psec = collapse(psec_arr)
            f.write("%s   %s    %s\n" % (aa, osec, psec))
print("1")

FileNotFoundError: [Errno 2] No such file or directory: '/cluster/gjb_lab/2472402/data/retr231/training/d3kxsa_.fasta'

# Evaluate PSSM layer 2

In [3]:
# first make sure that output path is valid, otherwise computation will go to waste
out_path = '/cluster/gjb_lab/2472402/outputs/pssm_cross_val/'
assert path.exists(out_path)
assert out_path[-1] == '/'


# path to training and validation examples
paths = ["/homes/adrozdetskiy/Projects/JnetDatasets/Jnet_training_output_v2/cross-val%d/" % num for num in range(1,8)]

# sequence dictionary or sd. need this to retrieve dssp information
sd = pickle.load(open('/cluster/gjb_lab/2472402/data/cross-val/cross_val_dict.pkl','rb')) 


# for each fold in the 7 fold cross validation procedure, do...
for counter, p in enumerate(paths):

    counter += 1 # start from 1
    
    valid_path = p + 'valid/'
    
    # get file names ending with .pssm
    valid_files = [f for f in os.listdir(valid_path) if f[-5:]=='.pssm']
    
    # read profile pssm as numpy arrays
    valid_pssm = [np.genfromtxt(fname = valid_path + fn) for fn in valid_files]
    
    # window profile pssms to get patterns
    # layer 1 sliding window flank = 8
    X_valid = [sliding_window(pssm, flank=8) for pssm in valid_pssm]
    del valid_pssm
    
    # get DSSP information 
    
    # remove .pssm extension and convert to int for lookup
    valid_numbers = [int(fn[:-5]) for fn in valid_files] 
    
    # lookup seqIDs given jnet number
    valid_dssp_IDs = [sd[sd.number == num].letters.values[0] for num in valid_numbers]
    
    # obtain 3-column DSSP of domains given their seqID. See get_dssp() defined above
    Y_valid = [get_dssp(ID) for ID in valid_dssp_IDs]

    assert all([y.shape[1] == 3 for y in Y_valid])
    assert all([x.shape[0] == y.shape[0] for (x, y) in zip(X_valid, Y_valid)])
    
    # generate layer 1 predictions
    
    model1 = keras.models.load_model('/cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold%d_model1' % counter)

    Y_pred_1 = [argmax(model1.predict(X)) for X in X_valid]
    
    # convert layer 1 predictions into layer 2 input
    X_valid_2 = [sliding_window(Y, flank = 9) for Y in Y_pred_1]
    
    # discard model object (and its input) from RAM
    del model1
    del Y_pred_1
    
    # sanity checks before passing into model2
    assert all([X.shape[1] == 57 for X in X_valid_2])
    assert all([X.dtype == 'float64' for X in X_valid_2])
    
    # generate layer 2 predictions
    
    model2 = keras.models.load_model('/cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold%d_model2' % counter)

    Y_pred_2 = [model2.predict(X) for X in X_valid_2]
    
    # discard model object (and its input) from RAM
    del model2
    del X_valid_2
    
    # open file connection
    out_file = out_path + 'fold%d' % counter + '_model2_scores.csv'
    with open(out_file, 'w') as f:
        # write header line
        f.write('seqID,Hcorrect,Htotal,Ecorrect,Etotal,Ccorrect,Ctotal,Ncorrect,Ntotal\n')

        # calculate accuracy using predictions and ground truth 
        for (num, pred, truth) in zip(valid_numbers, Y_pred_2, Y_valid):
            res_arr = get_Qall(pred,truth)
            f.write(str(num) + ',')
            res_arr.tofile(f, sep=',')
            f.write('\n')
    
    print('Results for cross validation fold %d are written to %s' % (counter, out_file))
    

Results for cross validation fold 1 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold1_model2_scores.csv
Results for cross validation fold 2 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold2_model2_scores.csv
Results for cross validation fold 3 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold3_model2_scores.csv
Results for cross validation fold 4 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold4_model2_scores.csv
Results for cross validation fold 5 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold5_model2_scores.csv
Results for cross validation fold 6 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold6_model2_scores.csv
Results for cross validation fold 7 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold7_model2_scores.csv


# Evaluate HMM layer 1 

In [3]:
# first make sure that output path is valid, otherwise computation will go to waste
out_path = '/cluster/gjb_lab/2472402/outputs/hmm_cross_val/'
assert path.exists(out_path)
assert out_path[-1] == '/'


# path to training and validation examples
paths = ["/homes/adrozdetskiy/Projects/JnetDatasets/Jnet_training_output_v2/cross-val%d/" % num for num in range(1,8)]

# sequence dictionary or sd. need this to retrieve dssp information
sd = pickle.load(open('/cluster/gjb_lab/2472402/data/cross-val/cross_val_dict.pkl','rb')) 


# for each fold in the 7 fold cross validation procedure, do...
for counter, p in enumerate(paths):

    counter += 1 # start from 1
    
    valid_path = p + 'valid/'
    
    # get file names ending with .hmm
    valid_files = [f for f in os.listdir(valid_path) if f[-4:]=='.hmm']
    
    # read profile hmms as numpy arrays
    valid_hmm = [np.genfromtxt(fname = valid_path + fn) for fn in valid_files]
    
    # window profile hmms to get patterns
    # layer 1 sliding window flank = 8
    X_valid = [sliding_window(hmm, flank=8) for hmm in valid_hmm]
    del valid_hmm
    
    # get DSSP information 
    
    # remove .hmm extension and convert to int for lookup
    valid_numbers = [int(fn[:-4]) for fn in valid_files] 
    
    # lookup seqIDs given jnet number
    valid_dssp_IDs = [sd[sd.number == num].letters.values[0] for num in valid_numbers]
    
    # obtain 3-column DSSP of domains given their seqID. See get_dssp() defined above
    Y_valid = [get_dssp(ID) for ID in valid_dssp_IDs]

    assert all([y.shape[1] == 3 for y in Y_valid])
    assert all([x.shape[0] == y.shape[0] for (x, y) in zip(X_valid, Y_valid)])
    
    # generate layer 1 predictions
    
    model1 = keras.models.load_model('/cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold%d_model1' % counter)

    Y_pred_1 = [argmax(model1.predict(X)) for X in X_valid]
        
    # open file connection
    out_file = out_path + 'fold%d' % counter + '_model1_scores.csv'
    with open(out_file, 'w') as f:
        # write header line
        f.write('seqID,Hcorrect,Htotal,Ecorrect,Etotal,Ccorrect,Ctotal,Ncorrect,Ntotal\n')

        # calculate accuracy using predictions and ground truth 
        for (num, pred, truth) in zip(valid_numbers, Y_pred_1, Y_valid):
            res_arr = get_Qall(pred,truth)
            f.write(str(num) + ',')
            res_arr.tofile(f, sep=',')
            f.write('\n')
    
    print('Results for cross validation fold %d are written to %s' % (counter, out_file))
    

Results for cross validation fold 1 are written to /cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold1_model1_scores.csv
Results for cross validation fold 2 are written to /cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold2_model1_scores.csv
Results for cross validation fold 3 are written to /cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold3_model1_scores.csv
Results for cross validation fold 4 are written to /cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold4_model1_scores.csv
Results for cross validation fold 5 are written to /cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold5_model1_scores.csv
Results for cross validation fold 6 are written to /cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold6_model1_scores.csv
Results for cross validation fold 7 are written to /cluster/gjb_lab/2472402/outputs/hmm_cross_val/fold7_model1_scores.csv


# Evaluate PSSM layer 1

In [4]:
# first make sure that output path is valid, otherwise computation will go to waste
out_path = '/cluster/gjb_lab/2472402/outputs/pssm_cross_val/'
assert path.exists(out_path)
assert out_path[-1] == '/'


# path to training and validation examples
paths = ["/homes/adrozdetskiy/Projects/JnetDatasets/Jnet_training_output_v2/cross-val%d/" % num for num in range(1,8)]

# sequence dictionary or sd. need this to retrieve dssp information
sd = pickle.load(open('/cluster/gjb_lab/2472402/data/cross-val/cross_val_dict.pkl','rb')) 


# for each fold in the 7 fold cross validation procedure, do...
for counter, p in enumerate(paths):

    counter += 1 # start from 1
    
    valid_path = p + 'valid/'
    
    # get file names ending with .pssm
    valid_files = [f for f in os.listdir(valid_path) if f[-5:]=='.pssm']
    
    # read profile pssm as numpy arrays
    valid_pssm = [np.genfromtxt(fname = valid_path + fn) for fn in valid_files]
    
    # window profile pssms to get patterns
    # layer 1 sliding window flank = 8
    X_valid = [sliding_window(pssm, flank=8) for pssm in valid_pssm]
    del valid_pssm
    
    # get DSSP information 
    
    # remove .pssm extension and convert to int for lookup
    valid_numbers = [int(fn[:-5]) for fn in valid_files] 
    
    # lookup seqIDs given jnet number
    valid_dssp_IDs = [sd[sd.number == num].letters.values[0] for num in valid_numbers]
    
    # obtain 3-column DSSP of domains given their seqID. See get_dssp() defined above
    Y_valid = [get_dssp(ID) for ID in valid_dssp_IDs]

    assert all([y.shape[1] == 3 for y in Y_valid])
    assert all([x.shape[0] == y.shape[0] for (x, y) in zip(X_valid, Y_valid)])
    
    # generate layer 1 predictions
    
    model1 = keras.models.load_model('/cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold%d_model1' % counter)

    Y_pred_1 = [argmax(model1.predict(X)) for X in X_valid]
        
    # open file connection
    out_file = out_path + 'fold%d' % counter + '_model1_scores.csv'
    with open(out_file, 'w') as f:
        # write header line
        f.write('seqID,Hcorrect,Htotal,Ecorrect,Etotal,Ccorrect,Ctotal,Ncorrect,Ntotal\n')

        # calculate accuracy using predictions and ground truth 
        for (num, pred, truth) in zip(valid_numbers, Y_pred_1, Y_valid):
            res_arr = get_Qall(pred,truth)
            f.write(str(num) + ',')
            res_arr.tofile(f, sep=',')
            f.write('\n')
    
    print('Results for cross validation fold %d are written to %s' % (counter, out_file))
    

Results for cross validation fold 1 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold1_model1_scores.csv
Results for cross validation fold 2 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold2_model1_scores.csv
Results for cross validation fold 3 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold3_model1_scores.csv
Results for cross validation fold 4 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold4_model1_scores.csv
Results for cross validation fold 5 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold5_model1_scores.csv
Results for cross validation fold 6 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold6_model1_scores.csv
Results for cross validation fold 7 are written to /cluster/gjb_lab/2472402/outputs/pssm_cross_val/fold7_model1_scores.csv
